In [1]:
import os
from transformers import TrainingArguments
from trl import DPOTrainer

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM


# Load model and tokenizer
model_id = "hatakeyama-llm-team/with_halcination_epoch4_ck10400"
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-29 17:03:17,128] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


In [2]:


train_dataset = load_dataset("hatakeyama-llm-team/dpo_data_for_tanuki", split="train")
eval_dataset = load_dataset("hatakeyama-llm-team/dpo_data_for_tanuki", split="test")

In [3]:


run_name = "0529_hatakeyama_with_halci_epoch4_dpo-lr5e-6-beta0.1"

os.environ["WANDB_PROJECT"] = "huggingface"
os.environ["WANDB_NAME"] = run_name

training_args = TrainingArguments(
    output_dir= f"./{run_name}",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=64,
    learning_rate=5.0e-7,
    warmup_ratio=0.1,
    num_train_epochs=5,
    logging_steps=1,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    remove_unused_columns=False,
    bf16=True,
    dataloader_num_workers=24,
    weight_decay = 0.0,
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,
    run_name=run_name,
    report_to="wandb",
    optim="adamw_torch",
)

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    loss_type="sigmoid",
    max_prompt_length=925,
    max_length=1150,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

Map: 100%|██████████| 1026/1026 [00:01<00:00, 758.00 examples/s]


In [4]:

dpo_trainer.train()
dpo_trainer.push_to_hub(f'hatakeyama-llm-team/{run_name}')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kanhatakeyama (weblab-geniac1). Use `wandb login --relogin` to force relogin


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'repetition_penalty': 1.15}
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

Step,Training Loss,Validation Loss
